In [ ]:
import os
os.getpid()

In [ ]:
import inspect
import hybridcuda
cures = hybridcuda.initcuda()
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
assert cures == 0

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

In [ ]:
class intrinsic_function_class(object):
    __hybrid_cuda__func__ = None
    __hybrid_cuda__intrinsic_function__ = ''
    def __init__(self, func, intrinsic):
        self.__hybrid_cuda__func__ = func
        self.__hybrid_cuda__intrinsic_function__ = intrinsic

    def __call__(self, *args):
        return self.__hybrid_cuda__func__(*args)


def intrinsic_function(intrinsicname):
    def decorator(func):
        return intrinsic_function_class(func, intrinsicname)
    return decorator

class intrinsic_constant_class(object):
    __hybrid_cuda__func__ = None
    __hybrid_cuda__intrinsic_constant__ = ''
    def __init__(self, func, intrinsic):
        self.__hybrid_cuda__func__ = func
        self.__hybrid_cuda__intrinsic_constant__ = intrinsic

    def __call__(self, *args):
        return self.__hybrid_cuda__func__(*args)


def intrinsic_constant(intrinsicname):
    def decorator(func):
        return intrinsic_constant_class(func, intrinsicname)
    return decorator

@intrinsic_function('__syncthreads')
def __syncthreads() -> None:
    pass


In [ ]:
@hybridfunction
def callsyncthreads():
    __syncthreads()

callsyncthreads[1,32]()

In [ ]:
@intrinsic_constant('42')
def answer() -> int:
    return 42

@hybridfunction
def callanswer(d):
    d[0] = answer()

d = [0]
callanswer[1,1](d)

assert d[0] == 42

### Grid Indices

In [ ]:
class threadIdx(object):
    __hybrid_cuda__intrinsic_type__ = "threadIdx"
    x : int = 0
    y : int = 0
    z : int = 0
        
class blockIdx(object):
    __hybrid_cuda__intrinsic_type__ = "blockIdx"
    x : int = 0
    y : int = 0
    z : int = 0
        
class blockDim(object):
    __hybrid_cuda__intrinsic_type__ = "blockDim"
    x : int = 1
    y : int = 1
    z : int = 1
        
class gridDim(object):
    __hybrid_cuda__intrinsic_type__ = "gridDim"
    x : int = 1
    y : int = 1
    z : int = 1

In [ ]:
@hybridfunction
def getthreadidx(d):
    d[0] = threadIdx.x + 42

d = [0]
getthreadidx[1,1](d)
assert d[0] == 42